In [61]:
# Import necessary libraries
import json
import os
from datetime import timedelta

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from pandas.tseries.offsets import MonthEnd
# Set plotting style
sns.set(style='whitegrid')

In [62]:
# Load the data from the text file
data = pd.read_csv('data/raw/PET/PET.txt', sep='\t', header=None, names=['json_str'])

# Function to parse JSON strings
def parse_json_str(json_str):
    return json.loads(json_str)

# Apply the function to parse the JSON strings
parsed_data = data['json_str'].apply(parse_json_str)

# Create a DataFrame from the parsed JSON data
df = pd.json_normalize(parsed_data)

# Display the first few rows of the DataFrame
df.head()

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,start,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[[20240617, 4.306], [20240610, 4.328], [202406...",NaN,NaN,NaN,NaN,NaN
1,PET.EMM_EPMPR_PTE_Y44HO_DPG.W,"Houston, TX Premium Reformulated Retail Gasoli...",Dollars per Gallon,W,$/gal,"Houston, TX Premium Reformulated Retail Gasoli...",None,"EIA, U.S. Energy Information Administration",USA-TX,USA-TX,20000605,20240617,2024-06-17T22:10:18-04:00,"[[20240617, 3.817], [20240610, 3.838], [202406...",NaN,NaN,NaN,NaN,NaN
2,PET.EMM_EPMMR_PTE_R5XCA_DPG.W,West Coast (PADD 5) Except California Midgrade...,Dollars per Gallon,W,$/gal,West Coast (PADD 5) Except California Midgrade...,None,"EIA, U.S. Energy Information Administration",NaN,USA-AK+USA-AZ+USA-HI+USA-NV+USA-OR+USA-WA,19980518,20240617,2024-06-17T22:10:18-04:00,"[[20240617, 4.137], [20240610, 4.306], [202406...",NaN,NaN,NaN,NaN,NaN
3,PET.EMM_EPMMR_PTE_Y05LA_DPG.W,"Los Angeles, CA Midgrade Reformulated Retail G...",Dollars per Gallon,W,$/gal,"Los Angeles, CA Midgrade Reformulated Retail G...",None,"EIA, U.S. Energy Information Administration",USA-CA,USA-CA,20000605,20240617,2024-06-17T22:10:18-04:00,"[[20240617, 4.865], [20240610, 4.913], [202406...",NaN,NaN,NaN,NaN,NaN
4,PET.EMM_EPMMR_PTE_Y05SF_DPG.W,"San Francisco, CA Midgrade Reformulated Retail...",Dollars per Gallon,W,$/gal,"San Francisco, CA Midgrade Reformulated Retail...",None,"EIA, U.S. Energy Information Administration",USA-CA,USA-CA,20000605,20240617,2024-06-17T22:10:18-04:00,"[[20240617, 5.002], [20240610, 5.106], [202406...",NaN,NaN,NaN,NaN,NaN


In [63]:
df.columns

Index(['series_id', 'name', 'units', 'f', 'unitsshort', 'description',
       'copyright', 'source', 'iso3166', 'geography', 'start', 'end',
       'last_updated', 'data', 'geography2', 'category_id',
       'parent_category_id', 'notes', 'childseries'],
      dtype='object')

In [64]:
# Explode the 'data' column to separate rows for each date-value pair
df = df.explode('data')
df.head(10)

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,start,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240617, 4.306]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240610, 4.328]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240603, 4.367]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240527, 4.412]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240520, 4.399]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240513, 4.434]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240506, 4.457]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240429, 4.469]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240422, 4.472]",NaN,NaN,NaN,NaN,NaN
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,W,$/gal,New York Harbor Premium Reformulated Retail Ga...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,20000605,20240617,2024-06-17T22:10:18-04:00,"[20240415, 4.224]",NaN,NaN,NaN,NaN,NaN


In [65]:
# Select distinct series_id, name, units
df_series = df[['series_id', 'name', 'units', 'unitsshort']].drop_duplicates()
df_series.head(10)

,series_id,name,units,unitsshort
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,$/gal
1,PET.EMM_EPMPR_PTE_Y44HO_DPG.W,"Houston, TX Premium Reformulated Retail Gasoli...",Dollars per Gallon,$/gal
2,PET.EMM_EPMMR_PTE_R5XCA_DPG.W,West Coast (PADD 5) Except California Midgrade...,Dollars per Gallon,$/gal
3,PET.EMM_EPMMR_PTE_Y05LA_DPG.W,"Los Angeles, CA Midgrade Reformulated Retail G...",Dollars per Gallon,$/gal
4,PET.EMM_EPMMR_PTE_Y05SF_DPG.W,"San Francisco, CA Midgrade Reformulated Retail...",Dollars per Gallon,$/gal
5,PET.EMM_EPM0R_PTE_R5XCA_DPG.W,West Coast (PADD 5) Except California All Grad...,Dollars per Gallon,$/gal
6,PET.EMM_EPM0R_PTE_Y05LA_DPG.W,"Los Angeles, CA All Grades Reformulated Retail...",Dollars per Gallon,$/gal
7,PET.EMM_EPM0R_PTE_Y05SF_DPG.W,"San Francisco, CA All Grades Reformulated Reta...",Dollars per Gallon,$/gal
8,PET.EMM_EPM0R_PTE_Y35NY_DPG.W,New York Harbor All Grades Reformulated Retail...,Dollars per Gallon,$/gal
9,PET.EMM_EPM0R_PTE_Y44HO_DPG.W,"Houston, TX All Grades Reformulated Retail Gas...",Dollars per Gallon,$/gal


In [66]:
# Filter rows based on columns: 'series_id', 'units'
df_series_nonas = df_series[(df_series['series_id'].notna()) & (df_series['units'].notna())]
df_series_nonas.head(10)

,series_id,name,units,unitsshort
0,PET.EMM_EPMPR_PTE_Y35NY_DPG.W,New York Harbor Premium Reformulated Retail Ga...,Dollars per Gallon,$/gal
1,PET.EMM_EPMPR_PTE_Y44HO_DPG.W,"Houston, TX Premium Reformulated Retail Gasoli...",Dollars per Gallon,$/gal
2,PET.EMM_EPMMR_PTE_R5XCA_DPG.W,West Coast (PADD 5) Except California Midgrade...,Dollars per Gallon,$/gal
3,PET.EMM_EPMMR_PTE_Y05LA_DPG.W,"Los Angeles, CA Midgrade Reformulated Retail G...",Dollars per Gallon,$/gal
4,PET.EMM_EPMMR_PTE_Y05SF_DPG.W,"San Francisco, CA Midgrade Reformulated Retail...",Dollars per Gallon,$/gal
5,PET.EMM_EPM0R_PTE_R5XCA_DPG.W,West Coast (PADD 5) Except California All Grad...,Dollars per Gallon,$/gal
6,PET.EMM_EPM0R_PTE_Y05LA_DPG.W,"Los Angeles, CA All Grades Reformulated Retail...",Dollars per Gallon,$/gal
7,PET.EMM_EPM0R_PTE_Y05SF_DPG.W,"San Francisco, CA All Grades Reformulated Reta...",Dollars per Gallon,$/gal
8,PET.EMM_EPM0R_PTE_Y35NY_DPG.W,New York Harbor All Grades Reformulated Retail...,Dollars per Gallon,$/gal
9,PET.EMM_EPM0R_PTE_Y44HO_DPG.W,"Houston, TX All Grades Reformulated Retail Gas...",Dollars per Gallon,$/gal


In [67]:
date_range_all = df.groupby('name')['end'].agg(['min', 'max'])
date_range_all_nonas = date_range_all[(date_range_all['min'].notna()) & (date_range_all['max'].notna())]
date_range_all_nonas.head(10)

,min,max
name,,
"\r\nU.S. Net Imports from Curacao of Liquified Petroleum Gases, Annual",2022,2022
"\r\nU.S. Net Imports from Curacao of Liquified Petroleum Gases, Monthly",202212,202212
"Alabama (with State Offshore) Associated-Dissolved Natural Gas, Reserves in Nonproducing Reservoirs, Wet, Annual",2021,2021
"Alabama (with State Offshore) Crude Oil Reserves in Nonproducing Reservoirs, Annual",2021,2021
"Alabama (with State Offshore) Natural Gas Liquids Lease Condensate, Reserves in Nonproducing Reservoirs, Annual",2021,2021
"Alabama (with State Offshore) Natural Gas Wet After Lease Separation, Reserves in Nonproducing Reservoirs, Annual",2021,2021
"Alabama (with State Offshore) Nonassociated Natural Gas, Reserves in Nonproducing Reservoirs, Wet, Annual",2021,2021
"Alabama Aviation Gasoline All Sales/Deliveries by Prime Supplier, Annual",2021,2021
"Alabama Aviation Gasoline All Sales/Deliveries by Prime Supplier, Monthly",202203,202203


In [68]:
# Return a count of the number of records by units 
df_series['units'].value_counts()

units
Thousand Barrels                     68599
Thousand Barrels per Day             67817
Dollars per Gallon                   17835
Thousand Gallons per Day             11784
Thousand Gallons                      6001
Barrels per Stream Day                1960
Million Barrels                       1261
Percent                                623
Barrels per Calendar Day               343
Dollars per Barrel                     304
Number of Elements                     287
Billion Cubic Feet                     168
Million Gallons                        117
Million Cubic Feet per Day              49
Short Tons per Day                      49
Thousand Barrels per Calendar Day       44
Thouand Barrels per Day                 32
Degree                                  32
Million Cubic Feet                      16
Thousand Feet                           14
Feet per Well                           12
Million Pounds                           7
Million Kilowatthours                    7
Thous

In [69]:
# Filter df to only include name containing 'Louisiana' and units in Dollars per Gallon
df_louisiana = df[df['name'].str.contains('Louisiana Total') & df['units'].str.contains('Dollars per Gallon')]
df_louisiana.head()

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,start,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202203, 3.09]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202202, 2.598]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202201, 2.331]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202112, 2.191]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202111, 2.332]",NaN,NaN,NaN,NaN,NaN


In [70]:
# Drop rows where 'data' is NaN or not a list
df_louisiana = df_louisiana.dropna(subset=['data'])
df_louisiana = df_louisiana[df_louisiana['data'].apply(lambda x: isinstance(x, list) and len(x) == 2)]
df_louisiana.head()

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,start,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202203, 3.09]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202202, 2.598]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202201, 2.331]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202112, 2.191]",NaN,NaN,NaN,NaN,NaN
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,198301,202203,2022-06-01T10:50:11-04:00,"[202111, 2.332]",NaN,NaN,NaN,NaN,NaN


In [71]:
# Split 'data' column into 'date' and 'value'
df_louisiana[['date', 'value']] = pd.DataFrame(df_louisiana['data'].tolist(), index=df_louisiana.index)
df_louisiana.head()

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,...,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries,date,value
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202203, 3.09]",NaN,NaN,NaN,NaN,NaN,202203,3.090
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202202, 2.598]",NaN,NaN,NaN,NaN,NaN,202202,2.598
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202201, 2.331]",NaN,NaN,NaN,NaN,NaN,202201,2.331
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202112, 2.191]",NaN,NaN,NaN,NaN,NaN,202112,2.191
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202111, 2.332]",NaN,NaN,NaN,NaN,NaN,202111,2.332


In [72]:
# Convert 'date' to datetime, coercing errors to NaT
df_louisiana['date'] = pd.to_datetime(df_louisiana['date'], format='%Y%m%d', errors='coerce')
# Convert 'value' to a numeric type, coercing errors to NaN
df_louisiana['value'] = pd.to_numeric(df_louisiana['value'], errors='coerce')
df_louisiana.head()

,series_id,name,units,f,unitsshort,description,copyright,source,iso3166,geography,...,end,last_updated,data,geography2,category_id,parent_category_id,notes,childseries,date,value
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202203, 3.09]",NaN,NaN,NaN,NaN,NaN,NaT,3.090
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202202, 2.598]",NaN,NaN,NaN,NaN,NaN,NaT,2.598
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202201, 2.331]",NaN,NaN,NaN,NaN,NaN,NaT,2.331
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202112, 2.191]",NaN,NaN,NaN,NaN,NaN,2021-01-02,2.191
93062,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,Dollars per Gallon,M,$/gal,Louisiana Total Gasoline Wholesale/Resale Pric...,None,"EIA, U.S. Energy Information Administration",USA-LA,USA-LA,...,202203,2022-06-01T10:50:11-04:00,"[202111, 2.332]",NaN,NaN,NaN,NaN,NaN,2021-01-01,2.332


In [73]:
df_louisiana.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4150 entries, 93062 to 116794
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   series_id           4150 non-null   object        
 1   name                4150 non-null   object        
 2   units               4150 non-null   object        
 3   f                   4150 non-null   object        
 4   unitsshort          4150 non-null   object        
 5   description         4150 non-null   object        
 6   copyright           4150 non-null   object        
 7   source              4150 non-null   object        
 8   iso3166             4150 non-null   object        
 9   geography           4150 non-null   object        
 10  start               4150 non-null   object        
 11  end                 4150 non-null   object        
 12  last_updated        4150 non-null   object        
 13  data                4150 non-null   object     

In [74]:
# Extract relevant columns and preprocess the data
df_louisiana['Date'] = pd.to_datetime(df_louisiana['date'])
df_louisiana['Price'] = df_louisiana['value']
df_louisiana = df_louisiana[['Date', 'Price', 'unitsshort', 'series_id', 'name', 'last_updated']].sort_values(by='Date').reset_index(drop=True)
df_louisiana.head()

,Date,Price,unitsshort,series_id,name,last_updated
0,1983-01-01,0.849,$/gal,PET.EMA_EPM0_PWA_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,2022-06-01T10:50:11-04:00
1,1983-01-01,0.940,$/gal,PET.EMA_EPM0_PTR_SLA_DPG.M,Louisiana Total Gasoline Through Company Outle...,2022-06-01T10:50:11-04:00
2,1983-01-01,0.940,$/gal,PET.EMA_EPM0_PTC_SLA_DPG.M,Louisiana Total Gasoline Through Company Outle...,2022-06-01T10:50:11-04:00
3,1983-01-01,0.898,$/gal,PET.EMA_EPM0_POR_SLA_DPG.M,Louisiana Total Gasoline Other End Users Price...,2022-06-01T10:50:11-04:00
4,1983-01-01,0.849,$/gal,PET.EMA_EPM0_PWG_SLA_DPG.M,Louisiana Total Gasoline Wholesale/Resale Pric...,2022-06-01T10:50:11-04:00


In [75]:
date_range_la = df_louisiana.groupby(['name', 'series_id'])['Date'].agg(['min', 'max'])
date_range_la.head(10)

,,min,max
name,series_id,,
"Louisiana Total Gasoline Bulk Sales Price by All Sellers, Annual",PET.EMA_EPM0_PBS_SLA_DPG.A,NaT,NaT
"Louisiana Total Gasoline Bulk Sales Price by All Sellers, Monthly",PET.EMA_EPM0_PBS_SLA_DPG.M,1994-01-01,2021-01-02
"Louisiana Total Gasoline DTW Sales Price by All Sellers, Annual",PET.EMA_EPM0_PDS_SLA_DPG.A,NaT,NaT
"Louisiana Total Gasoline DTW Sales Price by All Sellers, Monthly",PET.EMA_EPM0_PDS_SLA_DPG.M,1994-01-01,2021-01-02
"Louisiana Total Gasoline Other End Users Price by Refiners, Annual",PET.EMA_EPM0_POR_SLA_DPG.A,NaT,NaT
"Louisiana Total Gasoline Other End Users Price by Refiners, Monthly",PET.EMA_EPM0_POR_SLA_DPG.M,1983-01-01,2021-01-02
"Louisiana Total Gasoline Rack Sales Price by All Sellers, Annual",PET.EMA_EPM0_PRA_SLA_DPG.A,NaT,NaT
"Louisiana Total Gasoline Rack Sales Price by All Sellers, Monthly",PET.EMA_EPM0_PRA_SLA_DPG.M,1994-01-01,2021-01-02
"Louisiana Total Gasoline Retail Sales by All Sellers, Annual",PET.EMA_EPM0_PTA_SLA_DPG.A,NaT,NaT


In [76]:
# Return the table when both min and max are not equal to NaT 
date_range_la_nonas = date_range_la[(date_range_la['min'].notna()) & (date_range_la['max'].notna())]
date_range_la_nonas

,,min,max
name,series_id,,
"Louisiana Total Gasoline Bulk Sales Price by All Sellers, Monthly",PET.EMA_EPM0_PBS_SLA_DPG.M,1994-01-01,2021-01-02
"Louisiana Total Gasoline DTW Sales Price by All Sellers, Monthly",PET.EMA_EPM0_PDS_SLA_DPG.M,1994-01-01,2021-01-02
"Louisiana Total Gasoline Other End Users Price by Refiners, Monthly",PET.EMA_EPM0_POR_SLA_DPG.M,1983-01-01,2021-01-02
"Louisiana Total Gasoline Rack Sales Price by All Sellers, Monthly",PET.EMA_EPM0_PRA_SLA_DPG.M,1994-01-01,2021-01-02
"Louisiana Total Gasoline Retail Sales by All Sellers, Monthly",PET.EMA_EPM0_PTA_SLA_DPG.M,1983-01-01,2021-01-02
"Louisiana Total Gasoline Through Company Outlets Price by All Sellers, Monthly",PET.EMA_EPM0_PTC_SLA_DPG.M,1983-01-01,2021-01-02
"Louisiana Total Gasoline Through Company Outlets Price by Refiners, Monthly",PET.EMA_EPM0_PTR_SLA_DPG.M,1983-01-01,2021-01-02
"Louisiana Total Gasoline Wholesale/Resale Price by All Sellers, Monthly",PET.EMA_EPM0_PWA_SLA_DPG.M,1983-01-01,2021-01-02
"Louisiana Total Gasoline Wholesale/Resale Price by Refiners, Monthly",PET.EMA_EPM0_PWG_SLA_DPG.M,1983-01-01,2021-01-02


In [77]:
# Load and prepare the dataset
def load_gas_price_data(filepath):
    """
    Load and prepare the dataset from a raw text file containing JSON strings.
    
    Parameters:
    - filepath: Path to the .txt file containing the raw data.
    
    Returns:
    - A DataFrame with the data extracted from JSON strings, focusing on 'date' and 'value' columns.
    """
    # Load the data from the text file
    data = pd.read_csv(filepath, sep='\t', header=None, names=['json_str'])
    
    # Function to parse JSON strings
    def parse_json_str(json_str):
        return json.loads(json_str)
    
    # Apply the function to parse the JSON strings
    parsed_data = data['json_str'].apply(parse_json_str)
    
    # Create a DataFrame from the parsed JSON data
    df = pd.json_normalize(parsed_data)
    
    # Assuming 'date' and 'value' are keys in the JSON data, convert 'date' to datetime
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    
    # Keep only necessary columns and drop NA values
    #df = df[['date', 'value']].dropna()
    
    return df

# Prepare data for long format and additional transformations
def prepare_data(df, series_id="PET.EMA_EPM0_PBS_SLA_DPG.M"):
    """
    Prepare the dataset for analysis by performing several transformations.
    
    Parameters:
    - df: DataFrame to be transformed.
    - series_id: The series ID to filter the DataFrame by. Default is "PET.EMA_EPM0_PBS_SLA_DPG.M".
    
    Returns:
    - Transformed DataFrame.
    """
    # Filter based on series_id and non-NA 'units' column
    df = df[df['series_id'] == series_id]
    df = df.dropna(subset=['units'])
    
    # Explode the 'data' column to separate rows for each date-value pair
    df = df.explode('data')
    
    # Drop rows where 'data' is NaN or not a list
    df = df.dropna(subset=['data'])
    df = df[df['data'].apply(lambda x: isinstance(x, list) and len(x) == 2)]
    
    # Split 'data' column into 'date' and 'value'
    df[['date', 'value']] = pd.DataFrame(df['data'].tolist(), index=df.index)
    
    # Remove the 'data' column
    df = df.drop(columns=['data'])
    
    # Convert 'date' to datetime, coercing errors to NaT
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='coerce')
    
    # Convert 'value' to a numeric type, coercing errors to NaN
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    # Sort by 'date' to ensure chronological order
    df = df.sort_values('date')
    
    # Calculate log of 'value' and the difference in log_price
    df['log_price'] = np.log(df['value'])
    df['price_change'] = df['log_price'].diff()
    
    return df

# Function to perform AutoARIMA forecasting
def forecast_prices(df, cutoff_date):
    # Filter the DataFrame based on the cutoff date
    df_filtered = df[df['date'] < pd.to_datetime(cutoff_date)]
    
    # Convert prices to log prices to stabilize variance
    df_filtered['log_price'] = np.log(df_filtered['value'])
    
    # Define the model
    model = SARIMAX(df_filtered['log_price'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    
    # Fit the model
    results = model.fit()
    
    # Generate future dates
    future_dates = pd.date_range(df_filtered['date'].max() + MonthEnd(1), periods=13, freq='M')
    
    # Forecast future log prices
    forecast_log_prices = results.forecast(steps=13)
    
    # Convert log prices back to regular prices
    forecast_prices = np.exp(forecast_log_prices)
    
    # Create a DataFrame for the forecasted prices
    forecast_df = pd.DataFrame({
        'date': future_dates,
        'forecast_price': forecast_prices
    })
    
    return forecast_df

# Visualization function
def plot_forecast(df, forecast_df):
    base = alt.Chart(df).encode(
        x='date:T',
        y='value:Q'
    ).properties(
        width=700,
        height=600
    )

    line = base.mark_line(color='blue', size=3)
    points = base.mark_point(color='red')

    forecast_chart = alt.Chart(forecast_df).mark_line(color='green').encode(
        x='date:T',
        y='forecast_price:Q'
    )

    return line + points + forecast_chart

In [78]:
df = load_gas_price_data('data/raw/PET/PET.txt')
df

In [ ]:
df_prepared = prepare_data(df)
df_prepared.head()

In [ ]:
forecast_df = forecast_prices(df, '2021-01-01')
forecast_df.tail()

In [ ]:
plot_forecast(df, forecast_df)

In [ ]:
# # Save the model
# import joblib
# joblib.dump(model, 'models/oil_price_forecast_model.pkl')